# Harmony Service Chaining: PI 20.4 Demo

In PI 20.4, Harmony add service chaining capabilities to support requests that require functionality beyond that of a single service.
This notebook provides a basic workflow to demonstrate service chaining. For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this notebook can be found under the [docs/notebook-helpers](./notebook-helpers) folder.

## Prerequisites

1. Install Python 3. This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: pip install jupyterlab
3. Setup your ~/.netrc for Earthdata Login as described in Harmony API Introduction

## Set Up AWS

Once you have Zarr links, you can access them with your AWS credentials to the Harmony account.  Obtain the credentials and make sure your default AWS account uses them.  One way to do this is to edit `~/.aws/credentials` to have the following section:
```
[default]
aws_access_key_id = YOUR_HARMONY_ACCESS_KEY_ID
aws_secret_access_key = YOUR_HARMONY_SECRET_ACCESS_KEY
```
Restart your Jupyter kernel after completing this step

## Setup notebook environment and imports

We need to set up general-purpose imports and authentication

In [ ]:
%load_ext autoreload
%autoreload
%matplotlib inline

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt

# Import libraries used throughout the notebook
import json
from pprint import pprint
from time import sleep
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, print_async_status, show_shape


## Chained Services - PODAAC L2 Subsetter -> Harmony Zarr Converter

This request asks for spatial and variable subsetting of L2 data with output in the Zarr format. This requires chaining two services together, the PODAAC L2 Subsetter and the Harmony NetCDF to Zarr service.

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
asyncConfig = {
    'collection_id': 'C1234208438-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'mean_sea_surface',
    'format': 'application/x-zarr',
    'lat': '(-40:-10)',
    'lon': '(-50:0)',
    'time': '("2007-12-31T00:00:00.000Z":"2008-01-02T05:00:00.000Z")',
}

async_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?subset=lon{lon}&subset=lat{lat}&subset=time{time}&format={format}'.format(**asyncConfig)
async_response = get(async_url)

### Wait for results and get data links

In [ ]:
show_async_condensed(async_response, show_results=False)
job_results = get(async_response.url).json()
data_links = [link['href'] for link in job_results['links'] if link.get('rel') == 'data']


### Open a Zarr file

In [ ]:
import s3fs
import zarr

fs = s3fs.S3FileSystem(client_kwargs={'region_name':'us-west-2'})
store = fs.get_mapper(root=data_links[0], check=False)
zarr_file = zarr.open(store)

### Explore the Zarr file

In [ ]:
print(zarr_file.tree())

mss_array = zarr_file['mean_sea_surface']
time_array = zarr_file['time']

In [ ]:
from matplotlib import pyplot as plt
# Set up axis labels and title from metadata
plt.figure(figsize=(18, 7))
plt.ticklabel_format(style='plain', useOffset=False)
plt.title(mss_array.attrs['long_name'])
plt.xlabel(time_array.attrs['long_name'] )
plt.ylabel(mss_array.attrs['long_name'])

scale = mss_array.attrs['scale_factor']

# Plot the data
plt.plot(time_array, scale * mss_array[:])

## Perform the same request in Earthdata Search

https://search.uat.earthdata.nasa.gov/search/granules?p=C1234208438-POCLOUD&sb[0]=-40%2C-50%2C-10%2C0&qt=2007-12-31T00%3A00%3A00.000Z%2C2008-01-02T05%3A00%3A00.000Z
